In [1]:
import torch
import torch.nn as nn
import numpy as np
from ultralytics.nn.modules.conv import Conv, autopad
import math
import torch

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class Add(nn.Module):
    def __init__(self, ch, layer='pw_conv', reverse=False):
        super().__init__()
        self.reverse = reverse
        ch = ch[::-1] if self.reverse else ch

        ch_ = [[ch[i], ch[i+1]] for i in range(len(ch)-1)]
        
        self.layers = []
        for c1, c2, in ch_:
            if layer.lower() == 'conv':
                self.layers += [Conv(c1, c2, 3, 1, autopad(3))]
            elif layer.lower() == 'pw_conv':
                self.layers += [Conv(c1, c2, 1, 1, autopad(1))]
            elif layer.lower() == 'gpw_conv':
                self.layers += [Conv(c1, c2, 1, 1, autopad(1), g=math.gcd(c1, c2))]
        self.layers = nn.ModuleList(self.layers)


    def forward(self, x):
        x = x[::-1] if self.reverse else x

        y = x[0]
        for layer, xx in zip(self.layers, x[1:]):
            y = layer(y) + xx
        return y

In [17]:
class Add2(nn.Module):
    def __init__(self, ch, layer='pw_conv', reverse=False):
        super().__init__()
        self.reverse = reverse
        ch = ch[::-1] if self.reverse else ch

        self.feature_size = max(ch)
        # self.feature_size = int(np.mean(ch))

        self.layers = []

        for c in ch:
            if layer.lower() == 'conv':
                self.layers += [Conv(c, self.feature_size, 3, 1, autopad(3))]
            elif layer.lower() == 'pw_conv':
                self.layers += [Conv(c, self.feature_size, 1, 1, autopad(1))]
            elif layer.lower() == 'gpw_conv':
                self.layers += [Conv(c, self.feature_size, 3, 1, autopad(3), g=math.gcd(c, self.feature_size))]

        self.layers = nn.ModuleList(self.layers)

    def forward(self, x):
        x = x[::-1] if self.reverse else x

        y = self.layers[0](x[0])
        for layer, xx in zip(self.layers[1:], x[1:]):
            y += layer(xx)
        return y

In [18]:
def get_param(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [19]:
p1 = torch.zeros((1, 64, 40, 40))
p2 = torch.zeros((1, 128, 40, 40))

layer1 = nn.ModuleList([Add([64, 128]), Conv(128, 128, 3, 1)])
layer2 = [Add2([64, 128])]
layer2 += [Conv(layer2[0].feature_size, 128, 3, 1)]
layer2 = nn.ModuleList(layer2)

In [20]:
get_param(layer1)

156160

In [21]:
get_param(layer2)

172800

In [ ]:
from ultralytics import YOLO

model = YOLO('star2_downn-yolo12n.yaml').to('cpu')

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def get_param(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [3]:
# param add: 2650176, add2: 2646336
get_param(model)

2644768

In [1]:
import numpy as np

In [6]:
a = np.array([[1,2,3,4]]*3)

In [ ]:
1000/1.9118

620.3089138390918